# Calculations

## Importing

### Importing Relevant Packages

In [6]:
import numpy as np
import pandas as pd
import datetime

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa as tsa
from statsmodels.regression.rolling import RollingOLS

# to access Macro data
def FRED(code):
    fred_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id='
    df = pd.read_csv(fred_url + code)
    if 'DATE' in df.columns:
        df['DATE'] = df.apply(lambda row: datetime.date.fromisoformat(row.DATE), axis=1)
        df['DATE'] = pd.to_datetime(df['DATE'])
        df.set_index('DATE', inplace=True)
    time.sleep(2)  # So that I don't get blocked through excessive requests
    return df

# to calculate percent differences correctly
def pct_diff(series: pd.Series, periods: int = 1) -> pd.Series:
    """
    Calculate the percent difference of a pandas series.

    Parameters
    ----------
    series : pandas.Series
        Input series to calculate percent difference from.
    periods : int, default=1
        Number of periods to shift the series by.

    Returns
    -------
    pandas.Series
        Series with percent difference from the shifted series.

    """
    shifted_series = series.shift(periods)
    rs = series / shifted_series.where(shifted_series >= 0, np.nan) - 1
    return rs

# to quickly make a correlation matrix heatmap
def correlmap(data):
    data_correlation = data.corr()    
    sns.heatmap(data_correlation, vmin=-1, vmax=1, cmap='coolwarm', annot=True)
    plt.show()
    display(pd.DataFrame(data_correlation.sum(), columns=['Sum of Correlations']))

### Importing WRDS Data (S&P Compustat)

In [2]:
tdf = pd.read_csv('../Data/Thesis_Fundamentals_Data_1960-2023_FILTERED.csv',
                   dtype={
                       'gvkey': np.unicode_,
                       'datadate': np.unicode_,
                       'fyearq': 'float32',
                       'fqtr': 'float32',
                       'indfmt': np.unicode_,
                       'consol': np.unicode_,
                       'popsrc': np.unicode_,
                       'datafmt': np.unicode_,
                       'tic': np.unicode_,
                       'cusip': np.unicode_,
                       'conm': np.unicode_,
                       'curcdq': np.unicode_,
                       'datacqtr': np.unicode_,
                       'datafqtr': np.unicode_,
                       'naics': np.unicode_,
                       'sic': np.unicode_,
                       'ggroup': np.unicode_,
                       'gind': np.unicode_,
                       'gsector': np.unicode_,
                       'gsubind': np.unicode_,
                       'cik': np.unicode_,
                       'costat': np.unicode_,
                       'actq': 'float64',
                       'atq': 'float64',
                       'ceqq': 'float64',
                       'chq': 'float64',
                       'cshfdq': 'float64',
                       'cshoq': 'float64',
                       'cshprq': 'float64',
                       'dlcq': 'float64',
                       'dlttq': 'float64',
                       'dpactq': 'float64',
                       'dpq': 'float64',
                       'icaptq': 'float64',
                       'lctq': 'float64',
                       'niq': 'float64',
                       'oiadpq': 'float64',
                       'oibdpq': 'float64',
                       'piq': 'float64',
                       'revtq': 'float64',
                       'teqq': 'float64',
                       'txtq': 'float64',
                       'xintq': 'float64',
                       'xrdq': 'float64',
                       'dvy': 'float64',
                       'cik': 'float64',
                       'mktvalq': 'float64',
                       'prccq': 'float64', 
                       'tid': np.unicode_,
                       'qid': np.unicode_
                  }, index_col=0
                 )
tdf['datadate'] = pd.to_datetime(tdf.datadate)

print(len(tdf.gvkey.unique()), 'Firms')

28721 Firms


In [3]:
tdf.dtypes

gvkey               object
datadate    datetime64[ns]
fyearq             float32
fqtr               float32
indfmt              object
consol              object
popsrc              object
datafmt             object
tic                 object
cusip               object
conm                object
curcdq              object
datacqtr            object
datafqtr            object
actq               float64
atq                float64
ceqq               float64
chq                float64
cshfdq             float64
cshoq              float64
cshprq             float64
dlcq               float64
dlttq              float64
dpactq             float64
dpq                float64
icaptq             float64
lctq               float64
niq                float64
oiadpq             float64
oibdpq             float64
piq                float64
revtq              float64
teqq               float64
txtq               float64
xintq              float64
xrdq               float64
dvy                float64
c

In [4]:
def datacqtr_to_datadate(datacqtr):
    year = datacqtr[:4]
    q = datacqtr[-1]
    if q == '1':
        month = '03'
    elif q == '2':
        month = '06'
    elif q == '3':
        month = '09'
    else:
        month = '12'
    return np.datetime64(year + '-' + month) + np.timedelta64(1,'M') - np.timedelta64(1,'D')


tdf.datadate.fillna(tdf.datacqtr.apply(lambda q: datacqtr_to_datadate(q)), inplace=True)

C:\Users\jdsch\AppData\Local\Temp\ipykernel_14024\625245539.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tdf.datadate.fillna(tdf.datacqtr.apply(lambda q: datacqtr_to_datadate(q)), inplace=True)


### Importing Betas

In [8]:
betas = pd.read_csv(
    '../Data/Beta Data/Total Thesis Beta Data 2023-12-27.csv', 
    index_col=0,
    dtype={
        'Unnamed: 0': np.unicode_,
        'gvkey': np.unicode_,
        'iid': np.unicode_,
        'datadate': np.unicode_,
        'tic': np.unicode_,
        'cusip': np.unicode_,
        'conm': np.unicode_,
        'datafmt': np.unicode_,
        'tic': np.unicode_,
        'cusip': np.unicode_,
        'conm': np.unicode_,
        'cik': np.unicode_,
        'costat': np.unicode_,
        'ajpm': 'float64',
        'cheqvm': 'float64',
        'dvpspm': 'float64',
        'cshtrm': 'float64',
        'curcdm': np.unicode_,
        'prccm': 'float64',
        'trfm': 'float64',
        'trt1m': 'float64',
        'cshoq': 'float64',
        'cmth': 'float64',
        'cshom': 'float64',
        'cyear': 'float64',
        'secstat': np.unicode_,
        'cik': np.unicode_,
        'DGS1': 'float64',
        'market_return': 'float64',
        'occurences': 'int32',
        'stock_excess_return': 'float64',
        'market_excess_return': 'float64',
        'na_in_row': 'int32',
        'alpha': 'float64',
        'beta': 'float64',
        'alpha_se': 'float64',
        'beta_se': 'float64',
        'r2': 'float64'
    }
)
betas['datadate'] = pd.to_datetime(betas['datadate'])

In [9]:
betas.columns

Index(['gvkey', 'iid', 'datadate', 'tic', 'cusip', 'conm', 'ajpm', 'cheqvm',
       'dvpspm', 'cshtrm', 'curcdm', 'prccm', 'trfm', 'trt1m', 'cshoq', 'cmth',
       'cshom', 'cyear', 'secstat', 'cik', 'DGS1', 'market_return',
       'occurences', 'stock_excess_return', 'market_excess_return',
       'na_in_row', 'alpha', 'beta', 'alpha_se', 'beta_se', 'r2'],
      dtype='object')

#### Merging data with Compustat

In [10]:
tdf.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,mkvaltq,prccq,ggroup,gind,gsector,gsubind,naics,sic,tid,qid
001000_1966Q4,001000,1966-12-31,1966.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089,001000_1966-12-31,001000_1966Q4
001000_1967Q1,001000,1967-03-31,NaN,NaN,NaN,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089,NaN,001000_1967Q1
001000_1967Q2,001000,1967-06-30,NaN,NaN,NaN,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089,NaN,001000_1967Q2
001000_1967Q3,001000,1967-09-30,NaN,NaN,NaN,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089,NaN,001000_1967Q3
001000_1967Q4,001000,1967-12-31,1967.0,4.0,INDL,C,D,STD,AE.2,000032102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3089,001000_1967-12-31,001000_1967Q4


In [11]:
tdf.sort_values('datadate', inplace=True)
betas.sort_values('datadate', inplace=True)

tdf = pd.merge_asof(
    tdf.reset_index(), 
    betas[['gvkey','datadate','beta','alpha','beta_se','alpha_se','r2','DGS1']], 
    by='gvkey', 
    left_on='datadate',
    right_on='datadate',
    direction='nearest'
)

tdf.set_index('index', inplace=True)
tdf.index.name = None

tdf

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,naics,sic,tid,qid,beta,alpha,beta_se,alpha_se,r2,DGS1
006990_1961Q1,006990,1961-03-31,1961.0,1.0,INDL,C,D,STD,MHT.1,562876102,...,315,2300,006990_1961-03-31,006990_1961Q1,NaN,NaN,NaN,NaN,NaN,0.002403
007116_1961Q2,007116,1961-05-31,1961.0,1.0,INDL,C,D,STD,MAT,577081102,...,339930,3942,007116_1961-05-31,007116_1961Q2,NaN,NaN,NaN,NaN,NaN,0.002403
004390_1961Q2,004390,1961-05-31,1961.0,1.0,INDL,C,D,STD,EBF,293389102,...,323111,2761,004390_1961-05-31,004390_1961Q2,NaN,NaN,NaN,NaN,NaN,0.002403
005063_1961Q2,005063,1961-05-31,1961.0,1.0,INDL,C,D,STD,GIC.4,370120107,...,334220,3663,005063_1961-05-31,005063_1961Q2,NaN,NaN,NaN,NaN,NaN,0.002403
002102_1961Q2,002102,1961-05-31,1961.0,1.0,INDL,C,D,STD,4708C,074077934,...,311,2000,002102_1961-05-31,002102_1961Q2,NaN,NaN,NaN,NaN,NaN,0.002403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
009466_2023Q4,009466,2023-11-30,2023.0,2.0,INDL,C,D,STD,SCHL,807066105,...,513130,2731,009466_2023-11-30,009466_2023Q4,1.635395,0.013592,0.209902,0.013580,0.147264,0.004162
031846_2023Q4,031846,2023-11-30,2023.0,2.0,INDL,C,D,STD,DRI,237194105,...,722511,5812,031846_2023-11-30,031846_2023Q4,1.244205,0.000005,0.184076,0.010196,0.440623,0.004162
006669_2023Q4,006669,2023-11-30,2023.0,4.0,INDL,C,D,STD,LEN,526057104,...,236117,1531,006669_2023-11-30,006669_2023Q4,1.517115,0.012357,0.192239,0.010648,0.517795,0.004162
028195_2023Q4,028195,2023-11-30,2024.0,1.0,INDL,C,D,STD,JBL,466313103,...,334412,3672,028195_2023-11-30,028195_2023Q4,1.313116,0.017556,0.187106,0.010364,0.459222,0.004162


## Calculations

#### Notes on S&P Compustat terms

|Key|Definition|
|:---|:---|
|`gvkey`|S&P's Code|
|`datadate`|Quarter Ending Date|
|`fyearq`|Fiscal Year|
|`fqtr`|Fiscal Quarter|
|`indfmt`|Industrials Finances Format|
|`consol`|Subsidiaries Consolidated|
|`popsrc`|Domestic/International|
|`datafmt`|Standardized Quarterly Data|
|`tic`|Ticker (Symbol)|
|`conm`|Company Name|
|`curcdq`|Currency (USD)|
|`datacqtr`|Calendar Quarter (used in `qid`, main observation id)|
|`datafqtr`|Fiscal Quarter|
|`atq`|Total Assets|
|`actq`|Current Assets|
|`lctq`|Current Liabilities|
|`chq`|Cash|
|`dpq`|Accum. Depreciation and Amortization|
|`dlttq`|Total Long-Term Debt|
|`dlcq`|Debt in Current Liabilities|
|`dpactq`|Quarterly Depreciation|
|`icaptq`|Invested Cpaital|
|`niq`|Net Income|
|`piq`|Pretax Income|
|`oiadpq`|Quarterly Operating Income (EBIT)|
|`oibdpq`|Operating Income before Depreciation (EBITDA)|
|`txtq`|Income Taxes|
|`xintq`|Interest and Related Expense|
|`xrdq`|Research and Development Expense|
|`revtq`|Quarterly Revenue|
|`costat`|Company Active/Inactive|
|`prccq`|Stock Price at Quarter End|
|`cshfdq`|Diluted Shares|
|`cshoq`|Outstanding Shares|
|`cshprq`|Basic Shares|
|`naics`|NAICS Industry Code|
|`sic`|SIC Industry Code|

### Key Assumptions and Inputs

Window and Equity Risk Premium

I think a 20-quarter window and a 7% ERP should be fine.

In [12]:
# ----------------------------------------------------------------------------------------------------
# Rolling regression window
# ----------------------------------------------------------------------------------------------------
w = 20  # number of quarters we want the regressions to have in them


# ----------------------------------------------------------------------------------------------------
# Cost of Capital Inputs
# Equity Risk Premium
# ----------------------------------------------------------------------------------------------------
# risk-free rate will be determined above
#
# this is kind of arbitrary
# don't want to get too much into what the risk-free rate actually is
# I'll just use 7% and call it a day (for now!)
erp = 0.07

### Account Calculations

#### Single-period calculations

Single-row calculations

In [13]:
# Risk-Free Rate
# The DGS1 rate is for the month in which the beta was calculated
# We raise that rate to the power of (365.25 / days in the relevant month)
tdf['rfr'] = (1 + tdf['DGS1']) ** (365.25 / tdf.datadate.dt.day) - 1

# Market Capitalization
tdf['Market_Cap'] = tdf.cshfdq * tdf.prccq

# Cash
tdf['Cash'] = tdf.chq

# Debt
# Includes long-term debt and debt in current liabilities
tdf['Debt'] = tdf.dlttq + tdf.dlcq

# Net Debt
tdf['Net_Debt'] = tdf.Debt - tdf.Cash

# Total Long-Term Debt
tdf['Long_Term_Debt'] = tdf.dlttq

# Enterprise Value
tdf['Enterprise_Value'] = tdf.Market_Cap + tdf.Debt - tdf.Cash

# Current Assets
tdf['Current_Assets'] = tdf.actq

# Current Liabilities
tdf['Current_Liabilities'] = tdf.lctq

# Current Liabilities less ST Debt
tdf['Current_Liabilities_minus_ST_Debt'] = tdf.lctq - tdf.dlcq

# Current Ratio
tdf['Current_Ratio'] = tdf.Current_Assets / tdf.Current_Liabilities

# Percent Debt
tdf['Percent_Debt'] = tdf.Debt / (tdf.Enterprise_Value + tdf.Cash)

# Common Equity 
tdf['Common_Equity'] = tdf.ceqq

# Net Income
tdf['NI'] = tdf.niq

# Pretax Income
tdf['Pretax_Income'] = tdf.piq

# EBIT
tdf['EBIT'] = tdf.oiadpq

# EBITDA
tdf['EBITDA'] = tdf.oibdpq

# Taxes
tdf['Taxes'] = tdf.txtq

# Tax Rate
# Used in unlevered beta calculations and NOPAT calculations
# Here, I use the statutory tax rate for US businesses
# The statutory tax rate should approximate the marginal tax rate
# We should use the marginal tax rate in unlevering beta
# Source: https://taxfoundation.org/data/all/federal/historical-corporate-tax-rates-brackets/
def statutory_tax_rate(year):
    try:
        if year <= 1963:
            return 0.52
        elif year <= 1964:
            return 0.5
        elif year <= 1967:
            return 0.48
        elif year <= 1969:
            return 0.528
        elif year <= 1970:
            return 0.492
        elif year <= 1974:
            return 0.48
        elif year <= 1978:
            return 0.48
        elif year <= 1981:
            return 0.46
        elif year <= 1982:
            return 0.46
        elif year <= 1983:
            return 0.46
        elif year <= 1986:
            return 0.46
        elif year <= 1987:
            return 0.4
        elif year <= 1992:
            return 0.34
        elif year <= 2017:
            return 0.35
        else:
            return 0.21
    except:
        return np.nan
tdf['Tax_Rate'] = tdf.apply(lambda row: statutory_tax_rate(row.datadate.year), axis=1) 
#FORMULA FOR EFFECTIVE TAX RATE: tdf.txtq / tdf.piq

# Revenue
tdf['Revenue'] = tdf.revtq

# NOPAT
# EBIT after taxes
# I use the formula from the Corporate Finance Institute (they administer the CFA exams)
tdf['NOPAT'] = tdf.EBIT * (1 - tdf.Tax_Rate)

# Total Costs
tdf['Total_Costs'] = tdf.Revenue - tdf.EBIT

# Total Costs Including Taxes
tdf['Total_Costs_inc_t'] = tdf.Revenue - tdf.NOPAT

# R&D Expense
tdf['RnD'] = tdf.xrdq

# Assets
tdf['Assets'] = tdf.atq

# Invested Capital
tdf['Invested_Capital'] = tdf.icaptq

# Tobin's q
tdf['tobins_q'] = tdf.Enterprise_Value / tdf.Assets

# EBIT Margin
# I define this as  EBIT / Revenue
tdf['EBIT_margin'] = tdf.EBIT / tdf.Revenue

# Return on Invested Capital
# I define this as  EBIT * (1 - τ) / Invested Capital
tdf['ROIC'] = tdf.NOPAT / tdf.Invested_Capital
#tdf['ROIC_EBIT'] = tdf.EBIT / tdf.Invested_Capital

# Return on Assets
# I define this as  EBIT * (1 - τ) / Assets
# This is not the traditional ROA, which uses Net Income as the numerator
tdf['ROA'] = tdf.NOPAT / tdf.Assets

# Return on Equity
# I use the classic formula  Net Income / Equity
tdf['ROE'] = tdf.NI / tdf.Common_Equity

# Unlevered Beta
# I define this as  (Levered Beta) / (1 + (1 - τ) * (Net Debt / Market Cap))
tdf['beta_ul'] = tdf.beta / (1 + (1 - tdf.Tax_Rate) * (tdf.Net_Debt / tdf.Market_Cap))

# Rate of Required Return
# just a brute way of computing an upper bound for the firm's cost of capital
# if they were all equity funded, they should pay this rate on capital
# if they had debt that caused their WACC to cost more than this,
#     then they'd rather have all-equity financing and have this rate
tdf['RR'] = erp * tdf.beta_ul + tdf.rfr
tdf['qRR'] = (1 + tdf.RR) ** (1/4) - 1

# Required Return to Invested Capital
# A dollar value
# I define this as  (erp * beta_ul + rfr) * Invested Capital
tdf['RRIC'] = tdf.qRR * tdf.Invested_Capital

# Required Return to Enterprise Value
# A dollar value
# I define this as  (erp * beta_ul + rfr) * Enterprise Value
tdf['RREV'] = tdf.qRR * tdf.Enterprise_Value

# Excess Profit
# I define this as  EBIT * (1 - τ) - Required Return to Invested Capital
tdf['Excess_Profit'] = tdf.NOPAT - tdf.RRIC

# Total Costs Including Taxes and Required Return to Invested Capital
# I define this as  Revenue - Excess Profit
tdf['Total_Costs_inc_RRIC'] = tdf.Revenue - (tdf.NOPAT - tdf.RRIC)

# Total Costs Including Taxes and Required Return to Enterprise Value
# I define this as  Revenue - (EBIT * (1 - τ) - RREV)
tdf['Total_Costs_inc_RREV'] = tdf.Revenue - (tdf.NOPAT - tdf.RREV)


# Ratios
tdf['P_E'] = tdf.Market_Cap / tdf.NI
tdf['EV_EBIT'] = tdf.Enterprise_Value / tdf.EBIT
tdf['EV_EBITDA'] = tdf.Enterprise_Value / tdf.EBITDA
tdf['EV_S'] = tdf.Enterprise_Value / tdf.Revenue

#### Differences Calculations

In [14]:
accts = ['Market_Cap','Enterprise_Value','EBIT','Revenue','Total_Costs',
         'Total_Costs_inc_t','Assets','Invested_Capital','tobins_q','ROIC',
         'ROA','RRIC','RREV','Excess_Profit','Total_Costs_inc_RRIC']

tempdf = tdf.sort_values('datadate')[accts + ['gvkey']].groupby('gvkey').transform(pct_diff)

tdf = tdf.merge(tempdf, left_index=True, right_index=True, suffixes=('','_Growth'))

### Concentration Calculations

#### NAICS Categories

Make 1-, 2-, 3-, 4-, 5-, and 6-digit NAICS categories

In [15]:
def naics_trunc(naics, t):
    if naics != naics:
        return naics
    else:
        return naics[:t]

for i in range(1, 7):
    tdf['naics' + str(i)] = tdf['naics'].apply(lambda n: naics_trunc(n, i)).astype('object')

#### Calculate Market Share

By revenue

In [23]:
def mktshare(x):
    return x / x.sum()

for i in range(1, 7):
    mkt = tdf[['datacqtr','naics'+str(i),'Revenue']] \
    .groupby(['datacqtr','naics'+str(i)]).transform(mktshare)
    mkt.columns = ['mktshare' + str(i)]
    tdf = tdf.merge(mkt['mktshare' + str(i)], left_index=True, right_index=True, suffixes=('',''))
    del(mkt)

mkt = tdf[['datacqtr', 'gsubind', 'Revenue']].groupby(['datacqtr', 'gsubind']).transform(mktshare)
mkt.columns = ['mktshare_gsubind']
tdf = tdf.merge(mkt['mktshare_gsubind'], left_index=True, right_index=True, suffixes=('',''))
del(mkt)

#### Calculate Hirshman-Herfindahl Index

The Hirshman-Herfindahl Index is the weighted average market share, weighted by market shares.

$HHI = (\text{Market Share}_1)^2 + (\text{Market Share}_2)^2 + \cdots + (\text{Market Share}_n)^2$

By revenue

In [24]:
# function takes in a series or numpy array
def hirshherf(x):
    x /= x.sum()
    x = x * x
    return x.sum()

for i in range(1, 7):
    hhi = tdf[['datacqtr','naics'+str(i),'Revenue']] \
    .groupby(['datacqtr','naics'+str(i)]).transform(hirshherf)
    hhi.columns = ['hhi' + str(i)]
    tdf = tdf.merge(hhi['hhi' + str(i)], left_index=True, right_index=True, suffixes=('',''))
    del(hhi)

hhi = tdf[['datacqtr','gsubind','Revenue']].groupby(['datacqtr','gsubind']).transform(hirshherf)
hhi.columns = ['hhi_gsubind']
tdf = tdf.merge(hhi['hhi_gsubind'], left_index=True, right_index=True, suffixes=('',''))
del(hhi)

### Aggregate Calculations

Aggregated by quarter.

In [41]:
summation_items = [
    'Market_Cap',
    'Cash',
    'Debt',
    'Enterprise_Value',
    'EBIT',
    'Revenue',
    'NOPAT',
    'Total_Costs',
    'Total_Costs_inc_t',
    'Assets',
    'Invested_Capital'
]

summation_pivot = pd.pivot_table(tdf, values=summation_items, index='datacqtr', columns='gvkey')
summation_pivot.fillna(0, inplace=True)

agg = pd.DataFrame([summation_pivot[si].T.sum() for si in summation_items])
agg.index = summation_items
agg = agg.T
del(summation_pivot)

# Calculate RFR
agg['rfr'] = tdf[['datacqtr', 'rfr']].dropna().groupby('datacqtr').first()['rfr']

# Item calculations
agg['Net_Debt'] = agg.Debt - agg.Cash
agg['tobins_q_agg'] = agg.Enterprise_Value / agg.Assets
agg['ROIC_agg'] = agg.NOPAT / agg.Invested_Capital
agg['ROA_agg'] = agg.NOPAT / agg.Assets
agg['EBIT_Margin_agg'] = agg.EBIT / agg.Revenue
agg['RR_agg'] = erp + agg.rfr
agg['RRIC'] = agg.RR_agg * agg.Invested_Capital
agg['RREV'] = agg.RR_agg * agg.Enterprise_Value
agg['Excess_Profit'] = agg.NOPAT - agg.RRIC
agg['Total_Costs_inc_RRIC'] = agg.Revenue - (agg.NOPAT - agg.RRIC)
agg['Total_Costs_inc_RREV'] = agg.Revenue - (agg.NOPAT - agg.RREV)

#### Merge with Compustat

In [42]:
tdf = pd.merge(tdf, agg[['tobins_q_agg','ROIC_agg','ROA_agg','RR_agg']], left_on='datacqtr',right_index=True)

#### Additional Calculations based on Aggregate Data

In [43]:
tdf['π'] = tdf['ROIC'] / tdf['ROIC_agg']

In [44]:
del(betas)

### Rolling Regressions

Can change this to make it not a for loop and instead use groupby

Should be possible but I don't know how

**NOTE:** Whenever you add a column via a regression, add it's name to the part that checks for `inf` and `-inf`.

In [45]:
dfs = dict()

for i, ticker in enumerate(tdf.gvkey.unique()):
    
    dfs[ticker] = tdf[tdf.gvkey == ticker].copy()
    
    dfs[ticker].sort_values('datadate', inplace=True)
    
    # Price-Cost Margin
    try:
        pcm_mod = RollingOLS.from_formula('EBIT ~ Revenue', data=dfs[ticker], window=w)
        pcm_res = pcm_mod.fit()
        dfs[ticker]['price_cost_margin'] = pcm_res.params['Revenue']
        dfs[ticker]['price_cost_margin_se'] = pcm_res.bse['Revenue']
        dfs[ticker]['price_cost_margin_na'] = dfs[ticker][['Revenue','EBIT']].isnull().sum(axis=1).rolling(w).sum()
    except:
        dfs[ticker]['price_cost_margin'] = np.nan
        dfs[ticker]['price_cost_margin_se'] = np.nan
        dfs[ticker]['price_cost_margin_na'] = np.nan
        
    # Lerner Index
    try:
        pcm_mod = RollingOLS.from_formula('Excess_Profit ~ Revenue', data=dfs[ticker], window=w)
        pcm_res = pcm_mod.fit()
        dfs[ticker]['lerner'] = pcm_res.params['Revenue']
        dfs[ticker]['lerner_se'] = pcm_res.bse['Revenue']
        dfs[ticker]['lerner_na'] = dfs[ticker][['Revenue','Excess_Profit']].isnull().sum(axis=1).rolling(w).sum()
    except:
        dfs[ticker]['lerner'] = np.nan
        dfs[ticker]['lerner_se'] = np.nan
        dfs[ticker]['lerner_na'] = np.nan
    
    ## Panzar-Rosse H-Statistic
    ## First, with Revenue as the endogenous variable
    #try:
    #    hstat_mod = RollingOLS.from_formula('Revenue_Growth ~ Total_Costs_Growth', data=dfs[ticker], window=w)
    #    hstat_res = hstat_mod.fit()
    #    dfs[ticker]['simple_h'] = hstat_res.params['Total_Costs_Growth']
    #    dfs[ticker]['simple_h_se'] = hstat_res.bse['Total_Costs_Growth']
    #except:
    #    dfs[ticker]['simple_h'] = np.nan
    #    dfs[ticker]['simple_h_se'] = np.nan
    ## Second, with ROIC as the endogenous variable
    #try:
    #    hstat_roic_mod = RollingOLS.from_formula('ROIC ~ Total_Costs_Growth', data=dfs[ticker], window=w)
    #    hstat_roic_res = hstat_roic_mod.fit()
    #    dfs[ticker]['h_roic'] = hstat_roic_res.params['Total_Costs_Growth']
    #    dfs[ticker]['h_roic_se'] = hstat_roic_res.bse['Total_Costs_Growth']
    #except:
    #    dfs[ticker]['h_roic'] = np.nan
    #    dfs[ticker]['h_roic_se'] = np.nan
        
    # Marginal Return on Invested Capital (MROIC)
    # Supposed to be like the lerner index except with respect to invested capital
    try:
        mroic_mod = RollingOLS.from_formula('NOPAT ~ Invested_Capital', data=dfs[ticker], window=w)
        mroic_res = mroic_mod.fit()
        dfs[ticker]['mroic'] = mroic_res.params['Invested_Capital']
        dfs[ticker]['mroic_se'] = mroic_res.bse['Invested_Capital']
        dfs[ticker]['mroic_na'] = dfs[ticker][['NOPAT','Invested_Capital']].isnull().sum(axis=1).rolling(w).sum()
    except:
        dfs[ticker]['mroic'] = np.nan
        dfs[ticker]['mroic_se'] = np.nan
        dfs[ticker]['mroic_na'] = np.nan
        
    try:
        ar_mod = RollingOLS.from_formula('π ~ π.shift(1)', data=dfs[ticker], window=w)
        ar_res = ar_mod.fit()
        dfs[ticker]['ar_coef'] = ar_res.params['π.shift(1)']
        dfs[ticker]['ar_coef_se'] = ar_res.bse['π.shift(1)']
        dfs[ticker]['ar_int'] = ar_res.params['Intercept']
        dfs[ticker]['ar_int_se'] = ar_res.bse['Intercept']
        dfs[ticker]['ar_na'] = dfs[ticker][['π']].isnull().sum(axis=1).rolling(w).sum()
    except:
        dfs[ticker]['ar_coef'] = np.nan
        dfs[ticker]['ar_coef_se'] = np.nan
        dfs[ticker]['ar_int'] = np.nan
        dfs[ticker]['ar_int_se'] = np.nan
        dfs[ticker]['ar_na'] = np.nan
    
    # get rid of inf and -inf results
    # ADD ALL NEW COLUMNS HERE!!!
    reg_columns = ['price_cost_margin', 'lerner', 'mroic', 'ar_coef', 'ar_int']
    reg_columns = reg_columns + [reg_col + '_se' for reg_col in reg_columns]
    for reg_col in reg_columns:
        dfs[ticker][reg_col] = dfs[ticker][reg_col].astype('float64').replace([np.inf, -np.inf], np.nan)
    
    print(i, ticker)

    #if i > 50:
    #    break

c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)


0 006990
1 007116


c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = s

2 004390
3 005063
4 002102
5 003176


c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)
c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:255: RuntimeWarning: divide by zero encountered in scalar divide
  s2 = ssr / (nobs - tot_params)


...

26183 028788
26184 034496
26185 028035
26186 034067
26187 034413
26188 033200
26189 034065
26190 033214
26191 034445
26192 297507
26193 033600
26194 033598
26195 033447
26196 033597
26197 034066
26198 033810
26199 033233
26200 033528
26201 034467
26202 033781
26203 029473
26204 034448
26205 034449
26206 033599
26207 297434
26208 035614
26209 034420
26210 033595
26211 033225
26212 034429
26213 033133
26214 033823
26215 034421
26216 034418
26217 033779
26218 327451
26219 022072
26220 034482
26221 033240
26222 031295
26223 034469
26224 033780
26225 034291
26226 028368
26227 032561
26228 308989
26229 034016
26230 034278
26231 034015
26232 034221
26233 034287
26234 034048
26235 034307
26236 033989
26237 033136
26238 034220
26239 034003
26240 032941
26241 034039
26242 033137
26243 034242
26244 032677
26245 240066
26246 034270
26247 034266
26248 183302
26249 034361
26250 033011
26251 034410
26252 034064
26253 034193
26254 033859
26255 032901
26256 033185
26257 031402
26258 033861
26259 034400

c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:288: RuntimeWarning: invalid value encountered in matmul
  xpy = wx.T @ wy


27564 143297
27565 163058
27566 282960
27567 160864
27568 015399
27569 035019
27570 156214
27571 036615
27572 036603
27573 036462
27574 142932
27575 176751
27576 105689
27577 035345
27578 034753
27579 036748
27580 037711
27581 036691
27582 170879
27583 332967
27584 036688
27585 105646
27586 339950
27587 037750
27588 329579
27589 185281
27590 029769
27591 106003
27592 036700
27593 036649
27594 037673
27595 037773
27596 036648
27597 036609
27598 152290
27599 177887
27600 028415
27601 345556
27602 105950
27603 037797
27604 033541
27605 037325
27606 036525
27607 141781
27608 162514
27609 179929
27610 024277
27611 179903
27612 036515
27613 037350
27614 037034
27615 017396
27616 027668
27617 022362
27618 061633
27619 285660
27620 035541
27621 176133
27622 027519
27623 037386
27624 025472
27625 036506
27626 174302
27627 037364
27628 175230
27629 112211
27630 064203
27631 037013
27632 036565
27633 180081
27634 036536
27635 027813
27636 176992
27637 037198
27638 318525
27639 036537
27640 037268

c:\Users\jdsch\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\rolling.py:288: RuntimeWarning: invalid value encountered in matmul
  xpy = wx.T @ wy


28465 154795
28466 164699
28467 043585
28468 042848
28469 038022
28470 037428
28471 038126
28472 020214
28473 038885
28474 041474
28475 039964
28476 041569
28477 041153
28478 041220
28479 039991
28480 349705
28481 041878
28482 205876
28483 037707
28484 041885
28485 041897
28486 041636
28487 037725
28488 041687
28489 037820
28490 353945
28491 040650
28492 040116
28493 039874
28494 037998
28495 043555
28496 043507
28497 043521
28498 043551
28499 038790
28500 037513
28501 043439
28502 043435
28503 042243
28504 037669
28505 042284
28506 042070
28507 042890
28508 042891
28509 042892
28510 042911
28511 042912
28512 042847
28513 042033
28514 041953
28515 041958
28516 037928
28517 041981
28518 043197
28519 043198
28520 043236
28521 043220
28522 043134
28523 043137
28524 043186
28525 043195
28526 043382
28527 043385
28528 043386
28529 043289
28530 043286
28531 043287
28532 043128
28533 043026
28534 043038
28535 043057
28536 042950
28537 042987
28538 042988
28539 042984
28540 043100
28541 043113

## Saving

### Main Dataframe

The `tdf` dataframe was broken up in the previous step to run all the company regressions. Now, we bring it back together into `tdfs`. 

In [46]:
tdfs = pd.concat([dfs[ticker] for ticker in dfs.keys()])

In [47]:
try:
    del(dfs)
except:
    pass

try:
    del(tdf)
except:
    pass

### Saving the Data

In [48]:
tdfs.to_csv('../Data/regressions_results_' + str(datetime.datetime.today().date()) + '.csv')

In [49]:
tdfs

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,lerner_se,lerner_na,mroic,mroic_se,mroic_na,ar_coef,ar_coef_se,ar_int,ar_int_se,ar_na
006990_1961Q1,006990,1961-03-31,1961.0,1.0,INDL,C,D,STD,MHT.1,562876102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
006990_1961Q2,006990,1961-06-30,1961.0,2.0,INDL,C,D,STD,MHT.1,562876102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
006990_1961Q3,006990,1961-09-30,1961.0,3.0,INDL,C,D,STD,MHT.1,562876102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
006990_1961Q4,006990,1961-12-31,1961.0,4.0,INDL,C,D,STD,MHT.1,562876102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
006990_1962Q1,006990,1962-03-31,1962.0,1.0,INDL,C,D,STD,MHT.1,562876102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
038445_2023Q2,038445,2023-06-30,2023.0,2.0,INDL,C,D,STD,PSQH,693691107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
038445_2023Q3,038445,2023-09-30,2023.0,3.0,INDL,C,D,STD,PSQH,693691107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349994_2023Q2,349994,2023-07-31,2023.0,3.0,INDL,C,D,STD,CMND,185053402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
043651_2023Q3,043651,2023-08-31,2023.0,1.0,INDL,C,D,STD,WS,982104101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
